### Cross Validation、last入りデータ、パラメーター調整（dart）、カスタム評価関数

In [1]:
# lastとlast_diff を特徴量に追加
# 特徴量選択の数を調査した結果を反映
# カスタム評価関数
# パラメーター調整


In [2]:
import gc
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
import pandas_profiling as pp

book_name = '831'

warnings.filterwarnings('ignore')
%matplotlib inline

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)



In [3]:
%%time
h5File = "temp_data/h5/selectkbest_data_with_last.h5";
key_train = 'bestkselect_01/train';
key_valid = 'bestkselect_01/valid';
key_test = 'bestkselect_01/test';

train = pd.read_hdf(h5File, key_train);
valid = pd.read_hdf(h5File, key_valid);
test = pd.read_hdf(h5File, key_test);


CPU times: user 14.9 s, sys: 22.1 s, total: 37 s
Wall time: 43 s
CPU times: user 14.9 s, sys: 22.1 s, total: 37 s
Wall time: 43 s


In [4]:
gc.collect()

94

94

In [5]:
train

,P_2_min,P_2_max,P_2_avg,D_39_max,D_39_avg,B_1_min,B_1_max,B_1_avg,B_2_min,B_2_max,B_2_avg,R_1_min,R_1_max,R_1_avg,S_3_min,S_3_max,S_3_avg,D_41_min,D_41_max,D_41_avg,B_3_min,B_3_max,B_3_avg,D_42_min,D_42_max,D_42_avg,D_43_min,D_43_max,D_43_avg,D_44_min,D_44_max,D_44_avg,B_4_min,B_4_max,B_4_avg,D_45_min,D_45_max,D_45_avg,R_2_max,R_2_avg,D_46_min,D_46_max,D_46_avg,D_47_min,D_47_max,D_47_avg,D_48_min,D_48_max,D_48_avg,D_49_max,D_49_avg,B_7_min,B_7_max,B_7_avg,B_8_min,B_8_max,B_8_avg,D_51_min,D_51_max,D_51_avg,B_9_min,B_9_max,B_9_avg,R_3_min,R_3_max,R_3_avg,D_52_min,D_52_max,D_52_avg,P_3_min,P_3_max,P_3_avg,B_10_min,D_53_min,D_53_max,D_53_avg,B_11_min,B_11_max,B_11_avg,S_6_min,D_54_min,D_54_avg,R_4_max,R_4_avg,S_7_min,S_7_max,S_7_avg,S_8_min,S_8_avg,D_55_min,D_55_max,D_55_avg,D_56_min,D_56_max,D_56_avg,R_5_max,R_5_avg,D_58_min,D_58_max,D_58_avg,B_14_min,D_59_min,D_59_max,D_59_avg,D_60_max,D_61_min,D_61_max,D_61_avg,S_11_min,D_62_min,D_62_max,D_62_avg,D_64_cnt_O,D_64_cnt_U,D_65_max,B_16_min,B_16_max,B_16_avg,B_17_min,B_17_max,B_17_avg,B_18_min,B_18_max,B_18_avg,B_19_min,B_19_max,B_19_avg,B_20_min,B_20_max,B_20_avg,D_68_cnt_6,R_6_max,R_6_avg,S_13_min,S_13_avg,B_21_max,B_21_avg,B_22_min,B_22_max,B_22_avg,D_70_min,D_70_max,D_70_avg,D_72_max,D_72_avg,S_15_max,S_15_avg,B_23_min,B_23_max,B_23_avg,P_4_min,P_4_max,P_4_avg,D_74_min,D_74_max,D_74_avg,D_75_min,D_75_max,D_75_avg,B_24_max,B_24_avg,R_7_max,R_7_avg,D_77_min,D_77_max,D_77_avg,B_25_min,B_25_max,B_25_avg,D_78_max,D_78_avg,D_79_min,D_79_max,D_79_avg,R_8_max,R_8_avg,R_9_min,R_9_max,R_9_avg,R_10_max,R_10_avg,R_11_max,R_11_avg,D_81_max,D_81_avg,R_12_min,R_12_avg,B_28_min,B_28_max,B_28_avg,R_13_max,R_13_avg,R_15_max,R_15_avg,D_84_max,D_84_avg,R_16_max,R_16_avg,B_30_cnt_0,B_30_cnt_1,B_30_cnt_2,R_17_max,R_17_avg,B_31_min,B_31_avg,R_19_max,B_32_max,B_32_avg,S_20_max,S_20_avg,R_20_max,R_20_avg,R_21_max,R_21_avg,B_33_min,B_33_max,B_33_avg,D_89_max,D_89_avg,R_22_max,R_22_avg,D_91_max,D_91_avg,D_92_max,D_92_avg,R_24_max,R_24_avg,R_25_max,R_25_avg,S_22_max,S_22_avg,S_23_avg,S_24_max,S_24_avg,S_25_min,S_25_avg,D_103_min,D_103_max,D_103_avg,D_104_min,D_104_max,D_104_avg,D_107_min,D_107_max,D_107_avg,B_37_min,B_37_max,B_37_avg,R_26_min,R_26_max,R_26_avg,R_27_min,R_27_max,R_27_avg,B_38_cnt_1,B_38_cnt_2,B_38_cnt_4,B_38_cnt_5,B_38_cnt_6,B_38_cnt_7,D_112_min,D_112_max,D_112_avg,B_40_min,S_27_max,D_113_min,D_113_max,D_113_avg,D_114_cnt_0,D_114_cnt_1,D_115_min,D_115_max,D_115_avg,D_116_cnt_0,D_118_min,D_118_max,D_118_avg,D_119_min,D_119_max,D_119_avg,D_120_cnt_0,D_120_cnt_1,D_121_min,D_121_max,D_121_avg,D_122_min,D_122_max,D_122_avg,D_127_max,D_127_avg,D_128_min,D_128_max,D_128_avg,D_129_min,D_129_max,D_129_avg,B_41_max,D_130_min,D_130_max,D_130_avg,D_131_min,D_131_max,D_131_avg,D_132_max,D_133_max,D_133_avg,D_134_min,D_134_max,D_134_avg,D_136_min,D_136_max,D_136_avg,D_138_max,D_138_avg,D_139_min,D_139_max,D_139_avg,D_140_max,D_140_avg,D_141_min,D_141_max,D_141_avg,D_143_min,D_143_max,D_143_avg,D_145_min,D_145_max,D_145_avg,P_2_last,P_2_ldiff,D_39_last,D_39_ldiff,B_1_last,B_1_ldiff,B_2_last,B_2_ldiff,R_1_last,R_1_ldiff,S_3_last,D_41_last,D_41_ldiff,B_3_last,B_3_ldiff,D_42_last,D_43_last,D_44_last,D_44_ldiff,B_4_last,B_4_ldiff,D_45_last,R_2_last,R_2_ldiff,D_46_last,D_47_last,D_48_last,D_48_ldiff,D_49_last,B_7_last,B_7_ldiff,B_8_last,D_51_last,B_9_last,B_9_ldiff,R_3_last,D_52_last,P_3_last,P_3_ldiff,D_53_last,D_53_ldiff,B_11_last,B_11_ldiff,S_6_last,D_54_last,D_54_ldiff,R_4_last,R_4_ldiff,S_7_last,S_8_last,D_55_last,D_55_ldiff,D_56_last,R_5_last,R_5_ldiff,D_58_last,D_58_ldiff,D_59_last,D_61_last,D_61_ldiff,D_62_last,D_64_last_O,D_64_last_U,D_65_last,B_16_last,B_16_ldiff,B_17_last,B_18_last,B_18_ldiff,B_19_last,B_19_ldiff,B_20_last,B_20_ldiff,D_68_last_6,R_6_last,S_13_last,B_22_last,B_22_ldiff,D_70_last,D_72_last,D_72_ldiff,S_15_last,B_23_last,B_23_ldiff,P_4_last,P_4_ldiff,D_74_last,D_74_ldiff,D_75_last,D_75_ldiff,B_24_last,D_77_last,B_25_last,B_25_ldiff,D_78_last,D_78_ldiff,D_79_last,R_8_last,R_8_ldiff,R_9

,P_2_min,P_2_max,P_2_avg,D_39_max,D_39_avg,B_1_min,B_1_max,B_1_avg,B_2_min,B_2_max,B_2_avg,R_1_min,R_1_max,R_1_avg,S_3_min,S_3_max,S_3_avg,D_41_min,D_41_max,D_41_avg,B_3_min,B_3_max,B_3_avg,D_42_min,D_42_max,D_42_avg,D_43_min,D_43_max,D_43_avg,D_44_min,D_44_max,D_44_avg,B_4_min,B_4_max,B_4_avg,D_45_min,D_45_max,D_45_avg,R_2_max,R_2_avg,D_46_min,D_46_max,D_46_avg,D_47_min,D_47_max,D_47_avg,D_48_min,D_48_max,D_48_avg,D_49_max,D_49_avg,B_7_min,B_7_max,B_7_avg,B_8_min,B_8_max,B_8_avg,D_51_min,D_51_max,D_51_avg,B_9_min,B_9_max,B_9_avg,R_3_min,R_3_max,R_3_avg,D_52_min,D_52_max,D_52_avg,P_3_min,P_3_max,P_3_avg,B_10_min,D_53_min,D_53_max,D_53_avg,B_11_min,B_11_max,B_11_avg,S_6_min,D_54_min,D_54_avg,R_4_max,R_4_avg,S_7_min,S_7_max,S_7_avg,S_8_min,S_8_avg,D_55_min,D_55_max,D_55_avg,D_56_min,D_56_max,D_56_avg,R_5_max,R_5_avg,D_58_min,D_58_max,D_58_avg,B_14_min,D_59_min,D_59_max,D_59_avg,D_60_max,D_61_min,D_61_max,D_61_avg,S_11_min,D_62_min,D_62_max,D_62_avg,D_64_cnt_O,D_64_cnt_U,D_65_max,B_16_min,B_16_max,B_16_avg,B_17_min,B_17_max,B_17_avg,B_18_min,B_18_max,B_18_avg,B_19_min,B_19_max,B_19_avg,B_20_min,B_20_max,B_20_avg,D_68_cnt_6,R_6_max,R_6_avg,S_13_min,S_13_avg,B_21_max,B_21_avg,B_22_min,B_22_max,B_22_avg,D_70_min,D_70_max,D_70_avg,D_72_max,D_72_avg,S_15_max,S_15_avg,B_23_min,B_23_max,B_23_avg,P_4_min,P_4_max,P_4_avg,D_74_min,D_74_max,D_74_avg,D_75_min,D_75_max,D_75_avg,B_24_max,B_24_avg,R_7_max,R_7_avg,D_77_min,D_77_max,D_77_avg,B_25_min,B_25_max,B_25_avg,D_78_max,D_78_avg,D_79_min,D_79_max,D_79_avg,R_8_max,R_8_avg,R_9_min,R_9_max,R_9_avg,R_10_max,R_10_avg,R_11_max,R_11_avg,D_81_max,D_81_avg,R_12_min,R_12_avg,B_28_min,B_28_max,B_28_avg,R_13_max,R_13_avg,R_15_max,R_15_avg,D_84_max,D_84_avg,R_16_max,R_16_avg,B_30_cnt_0,B_30_cnt_1,B_30_cnt_2,R_17_max,R_17_avg,B_31_min,B_31_avg,R_19_max,B_32_max,B_32_avg,S_20_max,S_20_avg,R_20_max,R_20_avg,R_21_max,R_21_avg,B_33_min,B_33_max,B_33_avg,D_89_max,D_89_avg,R_22_max,R_22_avg,D_91_max,D_91_avg,D_92_max,D_92_avg,R_24_max,R_24_avg,R_25_max,R_25_avg,S_22_max,S_22_avg,S_23_avg,S_24_max,S_24_avg,S_25_min,S_25_avg,D_103_min,D_103_max,D_103_avg,D_104_min,D_104_max,D_104_avg,D_107_min,D_107_max,D_107_avg,B_37_min,B_37_max,B_37_avg,R_26_min,R_26_max,R_26_avg,R_27_min,R_27_max,R_27_avg,B_38_cnt_1,B_38_cnt_2,B_38_cnt_4,B_38_cnt_5,B_38_cnt_6,B_38_cnt_7,D_112_min,D_112_max,D_112_avg,B_40_min,S_27_max,D_113_min,D_113_max,D_113_avg,D_114_cnt_0,D_114_cnt_1,D_115_min,D_115_max,D_115_avg,D_116_cnt_0,D_118_min,D_118_max,D_118_avg,D_119_min,D_119_max,D_119_avg,D_120_cnt_0,D_120_cnt_1,D_121_min,D_121_max,D_121_avg,D_122_min,D_122_max,D_122_avg,D_127_max,D_127_avg,D_128_min,D_128_max,D_128_avg,D_129_min,D_129_max,D_129_avg,B_41_max,D_130_min,D_130_max,D_130_avg,D_131_min,D_131_max,D_131_avg,D_132_max,D_133_max,D_133_avg,D_134_min,D_134_max,D_134_avg,D_136_min,D_136_max,D_136_avg,D_138_max,D_138_avg,D_139_min,D_139_max,D_139_avg,D_140_max,D_140_avg,D_141_min,D_141_max,D_141_avg,D_143_min,D_143_max,D_143_avg,D_145_min,D_145_max,D_145_avg,P_2_last,P_2_ldiff,D_39_last,D_39_ldiff,B_1_last,B_1_ldiff,B_2_last,B_2_ldiff,R_1_last,R_1_ldiff,S_3_last,D_41_last,D_41_ldiff,B_3_last,B_3_ldiff,D_42_last,D_43_last,D_44_last,D_44_ldiff,B_4_last,B_4_ldiff,D_45_last,R_2_last,R_2_ldiff,D_46_last,D_47_last,D_48_last,D_48_ldiff,D_49_last,B_7_last,B_7_ldiff,B_8_last,D_51_last,B_9_last,B_9_ldiff,R_3_last,D_52_last,P_3_last,P_3_ldiff,D_53_last,D_53_ldiff,B_11_last,B_11_ldiff,S_6_last,D_54_last,D_54_ldiff,R_4_last,R_4_ldiff,S_7_last,S_8_last,D_55_last,D_55_ldiff,D_56_last,R_5_last,R_5_ldiff,D_58_last,D_58_ldiff,D_59_last,D_61_last,D_61_ldiff,D_62_last,D_64_last_O,D_64_last_U,D_65_last,B_16_last,B_16_ldiff,B_17_last,B_18_last,B_18_ldiff,B_19_last,B_19_ldiff,B_20_last,B_20_ldiff,D_68_last_6,R_6_last,S_13_last,B_22_last,B_22_ldiff,D_70_last,D_72_last,D_72_ldiff,S_15_last,B_23_last,B_23_ldiff,P_4_last,P_4_ldiff,D_74_last,D_74_ldiff,D_75_last,D_75_ldiff,B_24_last,D_77_last,B_25_last,B_25_ldiff,D_78_last,D_78_ldiff,D_79_last,R_8_last,R_8_ldiff,R_9

In [6]:
test

,P_2_min,P_2_max,P_2_avg,D_39_max,D_39_avg,B_1_min,B_1_max,B_1_avg,B_2_min,B_2_max,B_2_avg,R_1_min,R_1_max,R_1_avg,S_3_min,S_3_max,S_3_avg,D_41_min,D_41_max,D_41_avg,B_3_min,B_3_max,B_3_avg,D_42_min,D_42_max,D_42_avg,D_43_min,D_43_max,D_43_avg,D_44_min,D_44_max,D_44_avg,B_4_min,B_4_max,B_4_avg,D_45_min,D_45_max,D_45_avg,R_2_max,R_2_avg,D_46_min,D_46_max,D_46_avg,D_47_min,D_47_max,D_47_avg,D_48_min,D_48_max,D_48_avg,D_49_max,D_49_avg,B_7_min,B_7_max,B_7_avg,B_8_min,B_8_max,B_8_avg,D_51_min,D_51_max,D_51_avg,B_9_min,B_9_max,B_9_avg,R_3_min,R_3_max,R_3_avg,D_52_min,D_52_max,D_52_avg,P_3_min,P_3_max,P_3_avg,B_10_min,D_53_min,D_53_max,D_53_avg,B_11_min,B_11_max,B_11_avg,S_6_min,D_54_min,D_54_avg,R_4_max,R_4_avg,S_7_min,S_7_max,S_7_avg,S_8_min,S_8_avg,D_55_min,D_55_max,D_55_avg,D_56_min,D_56_max,D_56_avg,R_5_max,R_5_avg,D_58_min,D_58_max,D_58_avg,B_14_min,D_59_min,D_59_max,D_59_avg,D_60_max,D_61_min,D_61_max,D_61_avg,S_11_min,D_62_min,D_62_max,D_62_avg,D_64_cnt_O,D_64_cnt_U,D_65_max,B_16_min,B_16_max,B_16_avg,B_17_min,B_17_max,B_17_avg,B_18_min,B_18_max,B_18_avg,B_19_min,B_19_max,B_19_avg,B_20_min,B_20_max,B_20_avg,D_68_cnt_6,R_6_max,R_6_avg,S_13_min,S_13_avg,B_21_max,B_21_avg,B_22_min,B_22_max,B_22_avg,D_70_min,D_70_max,D_70_avg,D_72_max,D_72_avg,S_15_max,S_15_avg,B_23_min,B_23_max,B_23_avg,P_4_min,P_4_max,P_4_avg,D_74_min,D_74_max,D_74_avg,D_75_min,D_75_max,D_75_avg,B_24_max,B_24_avg,R_7_max,R_7_avg,D_77_min,D_77_max,D_77_avg,B_25_min,B_25_max,B_25_avg,D_78_max,D_78_avg,D_79_min,D_79_max,D_79_avg,R_8_max,R_8_avg,R_9_min,R_9_max,R_9_avg,R_10_max,R_10_avg,R_11_max,R_11_avg,D_81_max,D_81_avg,R_12_min,R_12_avg,B_28_min,B_28_max,B_28_avg,R_13_max,R_13_avg,R_15_max,R_15_avg,D_84_max,D_84_avg,R_16_max,R_16_avg,B_30_cnt_0,B_30_cnt_1,B_30_cnt_2,R_17_max,R_17_avg,B_31_min,B_31_avg,R_19_max,B_32_max,B_32_avg,S_20_max,S_20_avg,R_20_max,R_20_avg,R_21_max,R_21_avg,B_33_min,B_33_max,B_33_avg,D_89_max,D_89_avg,R_22_max,R_22_avg,D_91_max,D_91_avg,D_92_max,D_92_avg,R_24_max,R_24_avg,R_25_max,R_25_avg,S_22_max,S_22_avg,S_23_avg,S_24_max,S_24_avg,S_25_min,S_25_avg,D_103_min,D_103_max,D_103_avg,D_104_min,D_104_max,D_104_avg,D_107_min,D_107_max,D_107_avg,B_37_min,B_37_max,B_37_avg,R_26_min,R_26_max,R_26_avg,R_27_min,R_27_max,R_27_avg,B_38_cnt_1,B_38_cnt_2,B_38_cnt_4,B_38_cnt_5,B_38_cnt_6,B_38_cnt_7,D_112_min,D_112_max,D_112_avg,B_40_min,S_27_max,D_113_min,D_113_max,D_113_avg,D_114_cnt_0,D_114_cnt_1,D_115_min,D_115_max,D_115_avg,D_116_cnt_0,D_118_min,D_118_max,D_118_avg,D_119_min,D_119_max,D_119_avg,D_120_cnt_0,D_120_cnt_1,D_121_min,D_121_max,D_121_avg,D_122_min,D_122_max,D_122_avg,D_127_max,D_127_avg,D_128_min,D_128_max,D_128_avg,D_129_min,D_129_max,D_129_avg,B_41_max,D_130_min,D_130_max,D_130_avg,D_131_min,D_131_max,D_131_avg,D_132_max,D_133_max,D_133_avg,D_134_min,D_134_max,D_134_avg,D_136_min,D_136_max,D_136_avg,D_138_max,D_138_avg,D_139_min,D_139_max,D_139_avg,D_140_max,D_140_avg,D_141_min,D_141_max,D_141_avg,D_143_min,D_143_max,D_143_avg,D_145_min,D_145_max,D_145_avg,P_2_last,P_2_ldiff,D_39_last,D_39_ldiff,B_1_last,B_1_ldiff,B_2_last,B_2_ldiff,R_1_last,R_1_ldiff,S_3_last,D_41_last,D_41_ldiff,B_3_last,B_3_ldiff,D_42_last,D_43_last,D_44_last,D_44_ldiff,B_4_last,B_4_ldiff,D_45_last,R_2_last,R_2_ldiff,D_46_last,D_47_last,D_48_last,D_48_ldiff,D_49_last,B_7_last,B_7_ldiff,B_8_last,D_51_last,B_9_last,B_9_ldiff,R_3_last,D_52_last,P_3_last,P_3_ldiff,D_53_last,D_53_ldiff,B_11_last,B_11_ldiff,S_6_last,D_54_last,D_54_ldiff,R_4_last,R_4_ldiff,S_7_last,S_8_last,D_55_last,D_55_ldiff,D_56_last,R_5_last,R_5_ldiff,D_58_last,D_58_ldiff,D_59_last,D_61_last,D_61_ldiff,D_62_last,D_64_last_O,D_64_last_U,D_65_last,B_16_last,B_16_ldiff,B_17_last,B_18_last,B_18_ldiff,B_19_last,B_19_ldiff,B_20_last,B_20_ldiff,D_68_last_6,R_6_last,S_13_last,B_22_last,B_22_ldiff,D_70_last,D_72_last,D_72_ldiff,S_15_last,B_23_last,B_23_ldiff,P_4_last,P_4_ldiff,D_74_last,D_74_ldiff,D_75_last,D_75_ldiff,B_24_last,D_77_last,B_25_last,B_25_ldiff,D_78_last,D_78_ldiff,D_79_last,R_8_last,R_8_ldiff,R_9

,P_2_min,P_2_max,P_2_avg,D_39_max,D_39_avg,B_1_min,B_1_max,B_1_avg,B_2_min,B_2_max,B_2_avg,R_1_min,R_1_max,R_1_avg,S_3_min,S_3_max,S_3_avg,D_41_min,D_41_max,D_41_avg,B_3_min,B_3_max,B_3_avg,D_42_min,D_42_max,D_42_avg,D_43_min,D_43_max,D_43_avg,D_44_min,D_44_max,D_44_avg,B_4_min,B_4_max,B_4_avg,D_45_min,D_45_max,D_45_avg,R_2_max,R_2_avg,D_46_min,D_46_max,D_46_avg,D_47_min,D_47_max,D_47_avg,D_48_min,D_48_max,D_48_avg,D_49_max,D_49_avg,B_7_min,B_7_max,B_7_avg,B_8_min,B_8_max,B_8_avg,D_51_min,D_51_max,D_51_avg,B_9_min,B_9_max,B_9_avg,R_3_min,R_3_max,R_3_avg,D_52_min,D_52_max,D_52_avg,P_3_min,P_3_max,P_3_avg,B_10_min,D_53_min,D_53_max,D_53_avg,B_11_min,B_11_max,B_11_avg,S_6_min,D_54_min,D_54_avg,R_4_max,R_4_avg,S_7_min,S_7_max,S_7_avg,S_8_min,S_8_avg,D_55_min,D_55_max,D_55_avg,D_56_min,D_56_max,D_56_avg,R_5_max,R_5_avg,D_58_min,D_58_max,D_58_avg,B_14_min,D_59_min,D_59_max,D_59_avg,D_60_max,D_61_min,D_61_max,D_61_avg,S_11_min,D_62_min,D_62_max,D_62_avg,D_64_cnt_O,D_64_cnt_U,D_65_max,B_16_min,B_16_max,B_16_avg,B_17_min,B_17_max,B_17_avg,B_18_min,B_18_max,B_18_avg,B_19_min,B_19_max,B_19_avg,B_20_min,B_20_max,B_20_avg,D_68_cnt_6,R_6_max,R_6_avg,S_13_min,S_13_avg,B_21_max,B_21_avg,B_22_min,B_22_max,B_22_avg,D_70_min,D_70_max,D_70_avg,D_72_max,D_72_avg,S_15_max,S_15_avg,B_23_min,B_23_max,B_23_avg,P_4_min,P_4_max,P_4_avg,D_74_min,D_74_max,D_74_avg,D_75_min,D_75_max,D_75_avg,B_24_max,B_24_avg,R_7_max,R_7_avg,D_77_min,D_77_max,D_77_avg,B_25_min,B_25_max,B_25_avg,D_78_max,D_78_avg,D_79_min,D_79_max,D_79_avg,R_8_max,R_8_avg,R_9_min,R_9_max,R_9_avg,R_10_max,R_10_avg,R_11_max,R_11_avg,D_81_max,D_81_avg,R_12_min,R_12_avg,B_28_min,B_28_max,B_28_avg,R_13_max,R_13_avg,R_15_max,R_15_avg,D_84_max,D_84_avg,R_16_max,R_16_avg,B_30_cnt_0,B_30_cnt_1,B_30_cnt_2,R_17_max,R_17_avg,B_31_min,B_31_avg,R_19_max,B_32_max,B_32_avg,S_20_max,S_20_avg,R_20_max,R_20_avg,R_21_max,R_21_avg,B_33_min,B_33_max,B_33_avg,D_89_max,D_89_avg,R_22_max,R_22_avg,D_91_max,D_91_avg,D_92_max,D_92_avg,R_24_max,R_24_avg,R_25_max,R_25_avg,S_22_max,S_22_avg,S_23_avg,S_24_max,S_24_avg,S_25_min,S_25_avg,D_103_min,D_103_max,D_103_avg,D_104_min,D_104_max,D_104_avg,D_107_min,D_107_max,D_107_avg,B_37_min,B_37_max,B_37_avg,R_26_min,R_26_max,R_26_avg,R_27_min,R_27_max,R_27_avg,B_38_cnt_1,B_38_cnt_2,B_38_cnt_4,B_38_cnt_5,B_38_cnt_6,B_38_cnt_7,D_112_min,D_112_max,D_112_avg,B_40_min,S_27_max,D_113_min,D_113_max,D_113_avg,D_114_cnt_0,D_114_cnt_1,D_115_min,D_115_max,D_115_avg,D_116_cnt_0,D_118_min,D_118_max,D_118_avg,D_119_min,D_119_max,D_119_avg,D_120_cnt_0,D_120_cnt_1,D_121_min,D_121_max,D_121_avg,D_122_min,D_122_max,D_122_avg,D_127_max,D_127_avg,D_128_min,D_128_max,D_128_avg,D_129_min,D_129_max,D_129_avg,B_41_max,D_130_min,D_130_max,D_130_avg,D_131_min,D_131_max,D_131_avg,D_132_max,D_133_max,D_133_avg,D_134_min,D_134_max,D_134_avg,D_136_min,D_136_max,D_136_avg,D_138_max,D_138_avg,D_139_min,D_139_max,D_139_avg,D_140_max,D_140_avg,D_141_min,D_141_max,D_141_avg,D_143_min,D_143_max,D_143_avg,D_145_min,D_145_max,D_145_avg,P_2_last,P_2_ldiff,D_39_last,D_39_ldiff,B_1_last,B_1_ldiff,B_2_last,B_2_ldiff,R_1_last,R_1_ldiff,S_3_last,D_41_last,D_41_ldiff,B_3_last,B_3_ldiff,D_42_last,D_43_last,D_44_last,D_44_ldiff,B_4_last,B_4_ldiff,D_45_last,R_2_last,R_2_ldiff,D_46_last,D_47_last,D_48_last,D_48_ldiff,D_49_last,B_7_last,B_7_ldiff,B_8_last,D_51_last,B_9_last,B_9_ldiff,R_3_last,D_52_last,P_3_last,P_3_ldiff,D_53_last,D_53_ldiff,B_11_last,B_11_ldiff,S_6_last,D_54_last,D_54_ldiff,R_4_last,R_4_ldiff,S_7_last,S_8_last,D_55_last,D_55_ldiff,D_56_last,R_5_last,R_5_ldiff,D_58_last,D_58_ldiff,D_59_last,D_61_last,D_61_ldiff,D_62_last,D_64_last_O,D_64_last_U,D_65_last,B_16_last,B_16_ldiff,B_17_last,B_18_last,B_18_ldiff,B_19_last,B_19_ldiff,B_20_last,B_20_ldiff,D_68_last_6,R_6_last,S_13_last,B_22_last,B_22_ldiff,D_70_last,D_72_last,D_72_ldiff,S_15_last,B_23_last,B_23_ldiff,P_4_last,P_4_ldiff,D_74_last,D_74_ldiff,D_75_last,D_75_ldiff,B_24_last,D_77_last,B_25_last,B_25_ldiff,D_78_last,D_78_ldiff,D_79_last,R_8_last,R_8_ldiff,R_9

In [7]:
valid


customer_ID
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a    0.0
00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5    0.0
00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1    0.0
000041bdba6ecadd89a52d11886e8eaaec9325906c9723355abb5ca523658edc    0.0
00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8ad51ca8b8c4a24cefed    0.0
                                                                   ... 
e0dc8401f13e88ae5fadf2a6cd22ec8f2162a75964a00492ad406dfd437e716f    1.0
e0dcb868e734d5abe60f6a0d9107989aa0cef45c7da5f18642dee0b455c6c311    1.0
e0dcc67c6a5854ab27ac3b799141c2a5c9831471b93a96f1ee62c3e556133956    0.0
e0dd131940680fb03f52d808f3c1308abf4b057188fc0540614d2a543ab8a1ea    0.0
e0dd17ec1fae28e858808935b974ce142d53238d657018a1a2f129335e62d388    1.0
Name: target, Length: 458913, dtype: float64

customer_ID
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a    0.0
00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5    0.0
00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1    0.0
000041bdba6ecadd89a52d11886e8eaaec9325906c9723355abb5ca523658edc    0.0
00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8ad51ca8b8c4a24cefed    0.0
                                                                   ... 
e0dc8401f13e88ae5fadf2a6cd22ec8f2162a75964a00492ad406dfd437e716f    1.0
e0dcb868e734d5abe60f6a0d9107989aa0cef45c7da5f18642dee0b455c6c311    1.0
e0dcc67c6a5854ab27ac3b799141c2a5c9831471b93a96f1ee62c3e556133956    0.0
e0dd131940680fb03f52d808f3c1308abf4b057188fc0540614d2a543ab8a1ea    0.0
e0dd17ec1fae28e858808935b974ce142d53238d657018a1a2f129335e62d388    1.0
Name: target, Length: 458913, dtype: float64

In [8]:
X_train = train
y_train = valid
X_test = test

In [9]:
gc.collect()


0

0

In [10]:
import lightgbm as lgb
from typing import Tuple

def amex_metric(preds: np.ndarray, data: lgb.Dataset) -> Tuple[str, float, bool]:
# def amex_metric(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
#     print('preds', preds)
#     print('preds.shape', preds.shape)
#     print('data.get_label()', data.get_label())
#     print('data.get_label().shape', data.get_label().shape)

    y_pred = pd.DataFrame({'prediction': pd.Series(preds)})
    y_true = pd.DataFrame({'target': pd.Series(data.get_label())})

    def top_four_percent_captured(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns').sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
        
    def weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns').sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    def normalized_weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        y_true_pred = y_true.rename(columns={'target': 'prediction'})
        return weighted_gini(y_true, y_pred) / weighted_gini(y_true, y_true_pred)

#     df_y_true = 
    g = normalized_weighted_gini(y_true, y_pred)
    d = top_four_percent_captured(y_true, y_pred)
    m = 0.5 * (g + d)

    return 'amex_metric', m, True


In [11]:
%%time
from sklearn.model_selection import KFold, StratifiedKFold
import lightgbm as lgb

y_preds = []
models = []
oof_train = np.zeros((len(X_train),))
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

params = {
    'objective': 'binary',
    "metric" : "None",  # <= set `None` for custom metric
    'boosting': 'dart',
    'seed': 1,
    'num_leaves': 100,
    'learning_rate': 0.01,
    'feature_fraction': 0.20,
    'bagging_freq': 10,
    'bagging_fraction': 0.50,
    'n_jobs': -1,
    'lambda_l2': 2,
    'min_data_in_leaf': 40,
}

for fold_id, (train_index, test_index) in enumerate(skf.split(X_train, y_train)):
    gc.collect()

    X_tr , X_val = X_train.iloc[train_index], X_train.iloc[test_index]
    y_tr , y_val = y_train.iloc[train_index], y_train.iloc[test_index]

    print("============== fold_id:", fold_id)

    lgb_train = lgb.Dataset(X_tr, y_tr)
    lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)

    print("--- train ---")
    %time model = lgb.train(params, lgb_train,\
        valid_sets=[lgb_train, lgb_eval],\
        verbose_eval=10,\
        num_boost_round=1000,\
        early_stopping_rounds=10,\
        feval=amex_metric) # custom metrics

    print("--- oof_train ---")
    %time oof_train[test_index] = model.predict(X_val, num_iteration=model.best_iteration)
    print("--- y_pred ---")
    %time y_pred = model.predict(X_test, num_iteration=model.best_iteration)

    y_preds.append(y_pred)
    models.append(model)


============== fold_id: 0
--- train ---
============== fold_id: 0
--- train ---
[LightGBM] [Info] Number of positive: 95062, number of negative: 272068
[LightGBM] [Info] Number of positive: 95062, number of negative: 272068
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 1.184617 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 1.184617 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 118632
[LightGBM] [Info] Number of data points in the train set: 367130, number of used features: 500
[LightGBM] [Info] Total Bins 118632
[LightGBM] [Info] Number of data points in the train set: 367130, number of used features: 500
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258933 -> initscore=-1.051523
[LightGBM] [Info] Start training from score -1.051523
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.

[510]	training's amex_metric: 0.780607	valid_1's amex_metric: 0.767674
[520]	training's amex_metric: 0.780725	valid_1's amex_metric: 0.76766
[520]	training's amex_metric: 0.780725	valid_1's amex_metric: 0.76766
[530]	training's amex_metric: 0.781095	valid_1's amex_metric: 0.767798
[530]	training's amex_metric: 0.781095	valid_1's amex_metric: 0.767798
[540]	training's amex_metric: 0.781848	valid_1's amex_metric: 0.76835
[540]	training's amex_metric: 0.781848	valid_1's amex_metric: 0.76835
[550]	training's amex_metric: 0.782218	valid_1's amex_metric: 0.768618
[550]	training's amex_metric: 0.782218	valid_1's amex_metric: 0.768618
[560]	training's amex_metric: 0.782485	valid_1's amex_metric: 0.768646
[560]	training's amex_metric: 0.782485	valid_1's amex_metric: 0.768646
[570]	training's amex_metric: 0.782611	valid_1's amex_metric: 0.768339
[570]	training's amex_metric: 0.782611	valid_1's amex_metric: 0.768339
[580]	training's amex_metric: 0.782893	valid_1's amex_metric: 0.76893
[580]	train

[LightGBM] [Info] Total Bins 118632
[LightGBM] [Info] Number of data points in the train set: 367130, number of used features: 500
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258933 -> initscore=-1.051523
[LightGBM] [Info] Start training from score -1.051523
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258933 -> initscore=-1.051523
[LightGBM] [Info] Start training from score -1.051523
[10]	training's amex_metric: 0.75359	valid_1's amex_metric: 0.74293
[10]	training's amex_metric: 0.75359	valid_1's amex_metric: 0.74293
[20]	training's amex_metric: 0.759225	valid_1's amex_metric: 0.748965
[20]	training's amex_metric: 0.759225	valid_1's amex_metric: 0.748965
[30]	training's amex_metric: 0.761823	valid_1's amex_metric: 0.750585
[30]	training's amex_metric: 0.761823	valid_1's amex_metric: 0.750585
[40]	training's amex_metric: 0.762774	valid_1's amex_metric: 0.751616
[40]	training's amex_metric: 0.762774	valid_1's amex_metric: 0.751616
[50]	training's amex_metric: 0.764688	valid_1

[560]	training's amex_metric: 0.783084	valid_1's amex_metric: 0.767661
[570]	training's amex_metric: 0.783041	valid_1's amex_metric: 0.767745
[570]	training's amex_metric: 0.783041	valid_1's amex_metric: 0.767745
[580]	training's amex_metric: 0.783396	valid_1's amex_metric: 0.76794
[580]	training's amex_metric: 0.783396	valid_1's amex_metric: 0.76794
[590]	training's amex_metric: 0.783612	valid_1's amex_metric: 0.767839
[590]	training's amex_metric: 0.783612	valid_1's amex_metric: 0.767839
[600]	training's amex_metric: 0.783817	valid_1's amex_metric: 0.767984
[600]	training's amex_metric: 0.783817	valid_1's amex_metric: 0.767984
[610]	training's amex_metric: 0.784223	valid_1's amex_metric: 0.768127
[610]	training's amex_metric: 0.784223	valid_1's amex_metric: 0.768127
[620]	training's amex_metric: 0.784726	valid_1's amex_metric: 0.768513
[620]	training's amex_metric: 0.784726	valid_1's amex_metric: 0.768513
[630]	training's amex_metric: 0.785004	valid_1's amex_metric: 0.768799
[630]	tr

[30]	training's amex_metric: 0.76087	valid_1's amex_metric: 0.752192
[30]	training's amex_metric: 0.76087	valid_1's amex_metric: 0.752192
[40]	training's amex_metric: 0.761759	valid_1's amex_metric: 0.755342
[40]	training's amex_metric: 0.761759	valid_1's amex_metric: 0.755342
[50]	training's amex_metric: 0.763433	valid_1's amex_metric: 0.755662
[50]	training's amex_metric: 0.763433	valid_1's amex_metric: 0.755662
[60]	training's amex_metric: 0.764563	valid_1's amex_metric: 0.757083
[60]	training's amex_metric: 0.764563	valid_1's amex_metric: 0.757083
[70]	training's amex_metric: 0.765386	valid_1's amex_metric: 0.758601
[70]	training's amex_metric: 0.765386	valid_1's amex_metric: 0.758601
[80]	training's amex_metric: 0.765875	valid_1's amex_metric: 0.758557
[80]	training's amex_metric: 0.765875	valid_1's amex_metric: 0.758557
[90]	training's amex_metric: 0.766608	valid_1's amex_metric: 0.759052
[90]	training's amex_metric: 0.766608	valid_1's amex_metric: 0.759052
[100]	training's amex_

[610]	training's amex_metric: 0.783327	valid_1's amex_metric: 0.773428
[610]	training's amex_metric: 0.783327	valid_1's amex_metric: 0.773428
[620]	training's amex_metric: 0.784082	valid_1's amex_metric: 0.77331
[620]	training's amex_metric: 0.784082	valid_1's amex_metric: 0.77331
[630]	training's amex_metric: 0.78409	valid_1's amex_metric: 0.77335
[630]	training's amex_metric: 0.78409	valid_1's amex_metric: 0.77335
[640]	training's amex_metric: 0.784347	valid_1's amex_metric: 0.773263
[640]	training's amex_metric: 0.784347	valid_1's amex_metric: 0.773263
[650]	training's amex_metric: 0.7845	valid_1's amex_metric: 0.77369
[650]	training's amex_metric: 0.7845	valid_1's amex_metric: 0.77369
[660]	training's amex_metric: 0.784793	valid_1's amex_metric: 0.773754
[660]	training's amex_metric: 0.784793	valid_1's amex_metric: 0.773754
[670]	training's amex_metric: 0.785158	valid_1's amex_metric: 0.77418
[670]	training's amex_metric: 0.785158	valid_1's amex_metric: 0.77418
[680]	training's ame

[80]	training's amex_metric: 0.766107	valid_1's amex_metric: 0.756167
[80]	training's amex_metric: 0.766107	valid_1's amex_metric: 0.756167
[90]	training's amex_metric: 0.766202	valid_1's amex_metric: 0.756828
[90]	training's amex_metric: 0.766202	valid_1's amex_metric: 0.756828
[100]	training's amex_metric: 0.767114	valid_1's amex_metric: 0.757114
[100]	training's amex_metric: 0.767114	valid_1's amex_metric: 0.757114
[110]	training's amex_metric: 0.767314	valid_1's amex_metric: 0.75671
[110]	training's amex_metric: 0.767314	valid_1's amex_metric: 0.75671
[120]	training's amex_metric: 0.767987	valid_1's amex_metric: 0.757642
[120]	training's amex_metric: 0.767987	valid_1's amex_metric: 0.757642
[130]	training's amex_metric: 0.768316	valid_1's amex_metric: 0.757589
[130]	training's amex_metric: 0.768316	valid_1's amex_metric: 0.757589
[140]	training's amex_metric: 0.768926	valid_1's amex_metric: 0.758546
[140]	training's amex_metric: 0.768926	valid_1's amex_metric: 0.758546
[150]	traini

[660]	training's amex_metric: 0.785259	valid_1's amex_metric: 0.769774
[660]	training's amex_metric: 0.785259	valid_1's amex_metric: 0.769774
[670]	training's amex_metric: 0.785628	valid_1's amex_metric: 0.770298
[670]	training's amex_metric: 0.785628	valid_1's amex_metric: 0.770298
[680]	training's amex_metric: 0.785792	valid_1's amex_metric: 0.770621
[680]	training's amex_metric: 0.785792	valid_1's amex_metric: 0.770621
[690]	training's amex_metric: 0.78624	valid_1's amex_metric: 0.770579
[690]	training's amex_metric: 0.78624	valid_1's amex_metric: 0.770579
[700]	training's amex_metric: 0.786374	valid_1's amex_metric: 0.770549
[700]	training's amex_metric: 0.786374	valid_1's amex_metric: 0.770549
[710]	training's amex_metric: 0.786452	valid_1's amex_metric: 0.77096
[710]	training's amex_metric: 0.786452	valid_1's amex_metric: 0.77096
[720]	training's amex_metric: 0.786389	valid_1's amex_metric: 0.770923
[720]	training's amex_metric: 0.786389	valid_1's amex_metric: 0.770923
[730]	trai

[130]	training's amex_metric: 0.768359	valid_1's amex_metric: 0.759944
[130]	training's amex_metric: 0.768359	valid_1's amex_metric: 0.759944
[140]	training's amex_metric: 0.769159	valid_1's amex_metric: 0.760215
[140]	training's amex_metric: 0.769159	valid_1's amex_metric: 0.760215
[150]	training's amex_metric: 0.769364	valid_1's amex_metric: 0.759829
[150]	training's amex_metric: 0.769364	valid_1's amex_metric: 0.759829
[160]	training's amex_metric: 0.769711	valid_1's amex_metric: 0.759698
[160]	training's amex_metric: 0.769711	valid_1's amex_metric: 0.759698
[170]	training's amex_metric: 0.769974	valid_1's amex_metric: 0.760485
[170]	training's amex_metric: 0.769974	valid_1's amex_metric: 0.760485
[180]	training's amex_metric: 0.770139	valid_1's amex_metric: 0.760421
[180]	training's amex_metric: 0.770139	valid_1's amex_metric: 0.760421
[190]	training's amex_metric: 0.771041	valid_1's amex_metric: 0.761301
[190]	training's amex_metric: 0.771041	valid_1's amex_metric: 0.761301
[200]	

[710]	training's amex_metric: 0.786917	valid_1's amex_metric: 0.772527
[710]	training's amex_metric: 0.786917	valid_1's amex_metric: 0.772527
[720]	training's amex_metric: 0.787011	valid_1's amex_metric: 0.772198
[720]	training's amex_metric: 0.787011	valid_1's amex_metric: 0.772198
[730]	training's amex_metric: 0.787311	valid_1's amex_metric: 0.772637
[730]	training's amex_metric: 0.787311	valid_1's amex_metric: 0.772637
[740]	training's amex_metric: 0.787468	valid_1's amex_metric: 0.772771
[740]	training's amex_metric: 0.787468	valid_1's amex_metric: 0.772771
[750]	training's amex_metric: 0.787681	valid_1's amex_metric: 0.773204
[750]	training's amex_metric: 0.787681	valid_1's amex_metric: 0.773204
[760]	training's amex_metric: 0.788155	valid_1's amex_metric: 0.773066
[760]	training's amex_metric: 0.788155	valid_1's amex_metric: 0.773066
[770]	training's amex_metric: 0.788245	valid_1's amex_metric: 0.77301
[770]	training's amex_metric: 0.788245	valid_1's amex_metric: 0.77301
[780]	tr

In [12]:
gc.collect()

258

258

In [13]:
%%time
pd.DataFrame(oof_train).to_csv(f'work/oof_train_kfold_{book_name}.csv', index=False)

scores = [
    m.best_score['valid_1']['binary_logloss'] for m in models
]
score = sum(scores) / len(scores)
print('===CV scores===')
print(scores)
print(score)

KeyError: 'binary_logloss'

KeyError: 'binary_logloss'

In [14]:
gc.collect()

270

270

In [15]:
from sklearn.metrics import accuracy_score

y_pred_oof = (oof_train > 0.5).astype(int)
print('accuracy_score:', accuracy_score(y_train, y_pred_oof))


accuracy_score: 0.8983162385898852
accuracy_score: 0.8983162385898852


In [16]:
len(y_preds)

5

5

In [17]:
y_preds[0][:10]

array([0.13586917, 0.04043505, 0.0699157 , 0.35651589, 0.75560595,
       0.04050795, 0.84215619, 0.39218115, 0.70104432, 0.04171476])

array([0.13586917, 0.04043505, 0.0699157 , 0.35651589, 0.75560595,
       0.04050795, 0.84215619, 0.39218115, 0.70104432, 0.04171476])

In [18]:
y_sub = (sum(y_preds) / len(y_preds)).round(2)
# y_sub = (y_sub > 0.5).astype(int)
print(y_sub[:10])


[0.15 0.04 0.07 0.35 0.77 0.04 0.84 0.39 0.7  0.04]
[0.15 0.04 0.07 0.35 0.77 0.04 0.84 0.39 0.7  0.04]


In [19]:
sub = pd.read_csv('input/sample_submission.csv')
sub

,customer_ID,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0
...,...,...
924616,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,0
924617,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,0
924618,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...,0
924619,ffffddef1fc3643ea179c93245b68dca0f36941cd83977...,0


,customer_ID,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0
...,...,...
924616,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,0
924617,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,0
924618,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...,0
924619,ffffddef1fc3643ea179c93245b68dca0f36941cd83977...,0


In [20]:
y_pred

array([0.15856171, 0.04012441, 0.06893207, ..., 0.58664647, 0.3951241 ,
       0.15297886])

array([0.15856171, 0.04012441, 0.06893207, ..., 0.58664647, 0.3951241 ,
       0.15297886])

In [21]:
y_sub

array([0.15, 0.04, 0.07, ..., 0.6 , 0.38, 0.15])

array([0.15, 0.04, 0.07, ..., 0.6 , 0.38, 0.15])

In [22]:
# create submission
X_tiny = X_test.iloc[:,[0]]
X_tiny = pd.merge(X_tiny, sub, left_index=True, right_on='customer_ID')
X_tiny['prediction'] = y_sub
X_tiny = X_tiny.drop(columns=X_tiny.columns[[0]])
sub = X_tiny

In [23]:
sub

,customer_ID,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.15
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.04
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.07
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.35
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.77
...,...,...
924616,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,0.07
924617,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,0.77
924618,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...,0.60
924619,ffffddef1fc3643ea179c93245b68dca0f36941cd83977...,0.38


,customer_ID,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.15
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.04
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.07
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.35
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.77
...,...,...
924616,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,0.07
924617,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,0.77
924618,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...,0.60
924619,ffffddef1fc3643ea179c93245b68dca0f36941cd83977...,0.38


In [24]:
sub.to_csv('output/submission/sub_cross_validation_with_last_selectkbest_custom-metric_param-dart.csv', index=False)
